# Desafio Kinea


## Importando e definindo constantes globais

In [ ]:
from typing import Dict, List
from datetime import datetime

In [ ]:
from ambima_connect import AmbimaConnect
import utils

In [ ]:
import logging
logging.level=logging.INFO

In [ ]:
from titulos import LTN, NTNB, NTNF, LFT

## Interface com API Ambima

In [ ]:
data_list = AmbimaConnect('titulos publicos').content

In [ ]:
def clean_mercado_secundario(list_to_clean:List[Dict]):
  keep_keys = ['tipo_titulo','data_referencia','data_vencimento','taxa_indicativa','pu']
  res = [{key : val for key, val in sub.items() if key in keep_keys} for sub in list_to_clean]
  return res

In [ ]:
clean_mercado = clean_mercado_secundario(data_list)

## Cálculo Geral

In [ ]:
for titulo in clean_mercado:
  data_ref=datetime.strptime(titulo['data_referencia'], "%Y-%m-%d").date()
  data_venc=datetime.strptime(titulo['data_vencimento'], "%Y-%m-%d").date()

  if titulo['tipo_titulo'] == 'LTN':
    calc = LTN(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  elif titulo['tipo_titulo'] == 'NTN-B':
    calc = NTNB(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  elif titulo['tipo_titulo'] == 'NTN-F':
    calc = NTNF(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  elif titulo['tipo_titulo'] == 'LFT':
    calc = LFT(
      data_venc=data_venc,
      taxa=titulo['taxa_indicativa']/100,
      data_ref=data_ref,
      )

  titulo['pu_calc']=calc.pu
  titulo['pm_calc']=calc.pm
  titulo['duration_calc']=calc.duration

In [ ]:
t =[x for x in clean_mercado if x['tipo_titulo'] in ['NTN-F', 'NTN-B', 'LFT', 'LTN']]
print('Tipo, Data Venc, Real, Calc, Error')
for x in t: print(f"{x['tipo_titulo']}, {x['data_referencia']}, {x['pu']}, {x['pu_calc']}, {utils.check_error(x['pu_calc'], x['pu'])}")